In [ ]:
import scipy.io as sio, numpy as np
import os, re

In [55]:
folder_path = {"Long_words": "/home/tusharsingh/DATAs/speech_EEG/Long_words",
        "Short_Long_words": "/home/tusharsingh/DATAs/speech_EEG/Short_Long_words",
        "Short_words": "/home/tusharsingh/DATAs/speech_EEG/Short_words",
        "Vowels": "/home/tusharsingh/DATAs/speech_EEG/Vowels"}

In [78]:
def load_EEG(path):
    eeg = []
    labels = []
    patient_id = []
    for subject in os.scandir(path):
        if subject.is_file() and subject.name.endswith('.mat'):
            mat = sio.loadmat(subject.path)['eeg_data_wrt_task_rep_no_eog_256Hz_last_beep']
            for label in range(mat.shape[0]):
                for j in range(mat[label].shape[0]):
                    eeg.append(mat[label][j][:64,:])
                    labels.append(label)
                    patient_id.append(int(re.search("[0-9]+", re.match("sub_[0-9]+[a-z]*_", subject.name).group()).group()))
    return eeg,labels,patient_id